In [1]:
import io
import bson
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.data import imread
import multiprocessing as mp
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [8]:
data = bson.decode_file_iter(open('train_example.bson', 'rb'))
ids = []
pics = []
prod_to_category = dict()

for c, d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    prod_to_category[product_id] = category_id
    for e, pic in enumerate(d['imgs']):
        ids.append(d['category_id'])
        pics.append(imread(io.BytesIO(pic['picture'])))
        picture = imread(io.BytesIO(pic['picture']))
        # do something with the picture, etc

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

ids = np.asarray(ids)

nppics = np.asarray(pics)

In [9]:
enc = OneHotEncoder()
onehot_ids = enc.fit_transform(ids.reshape(-1, 1))

In [20]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_4x4(x):
    return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 3, 3, 1], padding='SAME')

#set up placeholders
x = tf.placeholder(tf.float32, shape=[None, 180, 180, 3], name='input_images')
y_ = tf.placeholder(tf.float32, shape=[None, 36], name='one_hotted_labels')

#set up neurons for 1st convolutional layer
W_conv1 = weight_variable([5, 5, 3, 16])
b_conv1 = bias_variable([16])

#don't know why this step is necessary, but it is
x_image = tf.reshape(x, [-1,180,180,3])

#do the first convolutional layer, pooling
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_4x4(h_conv1)

#do the second convolutional layer, pooling
W_conv2 = weight_variable([45, 45, 16, 32]) #45x45 images, 16 input channels, 32 output channels
b_conv2 = bias_variable([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_3x3(h_conv2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 15*15*32])

W = weight_variable([15*15*32, 36])
b = bias_variable([36])

y_conv = tf.matmul(h_pool2_flat, W) + b

sess = tf.InteractiveSession()

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))


train_step = tf.train.AdamOptimizer(3e-38).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
#Do the training
for _ in range(3):
    batch = nppics
    xvals = nppics
    yvals = onehot_ids
    
    print(xvals.shape)
    print(yvals.shape)
    train_step.run(feed_dict={x: xvals, y_: yvals.todense()})

(110, 180, 180, 3)
(110, 36)
(110, 180, 180, 3)
(110, 36)
(110, 180, 180, 3)
(110, 36)


In [12]:
type(yvals)

scipy.sparse.csr.csr_matrix

In [13]:
yvals.shape

(110, 36)

In [15]:
yvals.todense()

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])